## How to Query satellittdata.no Using CSW

The **Catalogue Service for the Web (CSW)** is an **Open Geospatial Consortium (OGC)** standard that enables structured querying and discovery of geospatial metadata over the web. It provides a standardised interface to search metadata catalogues using parameters such as keywords, spatial extent, and time. This tutorial demonstrates how to use CSW to query datasets hosted on [satellittdata.no](https://www.satellittdata.no/), Norway’s national satellite data portal. With Python tools, we can efficiently discover and retrieve metadata about available Earth observation products.

## Importing libraries

Let's begin by importing the Python libraries we’ll use in this session.

If you don’t already have **OWSLib** installed, you may need to run:

```bash
pip install owslib
```

Now we can import the libraries

In [1]:
from owslib import fes
from owslib.csw import CatalogueServiceWeb
from datetime import datetime, timedelta
import pytz

## Defining functions

Let's now define some functions that will help us use CSW to query. We will use these functions to perform a range of queries in the next section.

In [2]:
def _get_csw_connection(endpoint):
    """ Connect to CSW server
    """
    csw = CatalogueServiceWeb(endpoint, timeout=60)
    return csw

def _get_freetxt_search(kw_names):
    """
    Retuns a CSW search object based on input string
    """
    freetxt_filt = fes.PropertyIsLike('apiso:AnyText',  literal=('%s' % kw_names),
                                      escapeChar="\\", singleChar=".",
                                      wildCard="*", matchCase="True")
    return freetxt_filt

def _get_csw_records(csw, filter_list, pagesize=10, maxrecords=1):
    """
    Iterate `maxrecords`/`pagesize` times until the requested value in
    `maxrecords` is reached.
    """
    csw_records = {}
    startposition = 0
    nextrecord = getattr(csw, "results", 1)
    while nextrecord != 0:
        csw.getrecords2(
            constraints=filter_list,
            startposition=startposition,
            maxrecords=pagesize,
            outputschema="http://www.opengis.net/cat/csw/2.0.2",
            esn='full',
        )
        print(csw.results)
        csw_records.update(csw.records)
        if csw.results["nextrecord"] == 0:
            break
        startposition += pagesize + 1  # Last one is included.
        if startposition >= maxrecords:
            break
    csw.records.update(csw_records)

def _fes_date_filter(start, stop, constraint="overlaps"):
    """
    Take datetime-like objects and returns a fes filter for date range
    (begin and end inclusive).
    NOTE: Truncates the minutes!!!
    """
    start = start.strftime("%Y-%m-%d %H:00")
    stop = stop.strftime("%Y-%m-%d %H:00")
    if constraint == "overlaps":
        propertyname = "apiso:TempExtent_begin"
        begin = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname, literal=stop)
        propertyname = "apiso:TempExtent_end"
        end = fes.PropertyIsGreaterThanOrEqualTo(
            propertyname=propertyname, literal=start
        )
    elif constraint == "within":
        propertyname = "apiso:TempExtent_begin"
        begin = fes.PropertyIsGreaterThanOrEqualTo(
            propertyname=propertyname, literal=start
        )
        propertyname = "apiso:TempExtent_end"
        end = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname, literal=stop)
    else:
        raise NameError("Unrecognized constraint {}".format(constraint))
    return begin, end

## Querying using CSW

Let's first define the parameters we want to use for querying.

In [5]:
##### Main start here ##############
endpoint='https://nbs.csw.met.no'
bbox = [-80, 60.00, 10, 90.00]
start = datetime(2025, 3, 5, 00, 00, 00).replace(tzinfo=pytz.utc)
stop = datetime(2025, 6, 15, 00, 00, 00).replace(tzinfo=pytz.utc)
kw_names = 'Sentinel-1'
#crs='urn:ogc:def:crs:OGC:1.3:CRS84'



crs='urn:x-ogc:def:crs:EPSG:6.18:4326'

constraints = []
# connect to endpoint
try:
    csw = _get_csw_connection(endpoint)
except Exception as e:
    print("Exception: %s" % str(e))

if kw_names:
    freetxt_filt = _get_freetxt_search(kw_names)
    constraints.append(freetxt_filt)

if all(v is not None for v in [start, stop]):
    begin, end = _fes_date_filter(start, stop)
    constraints.append(begin)
    constraints.append(end)

if bbox:
     bbox_crs = fes.BBox(bbox, crs=crs)
     constraints.append(bbox_crs)

if len(constraints) >= 2:
    filter_list = [fes.And(constraints)]
else:
    filter_list = constraints


_get_csw_records(csw, filter_list, pagesize=10, maxrecords=100)
url_opendap = []

#print(len(csw.records))
for key, value in list(csw.records.items()):
    for ref in value.references:
        print(ref)
        if ref['scheme'] == 'OPeNDAP:OPeNDAP':
            #if '/2024/' in ref['url']:
            #print(ref['url'])
            url_opendap.append(ref['url'])

{'matches': 44, 'returned': 10, 'nextrecord': 11}
{'matches': 44, 'returned': 10, 'nextrecord': 21}
{'matches': 44, 'returned': 10, 'nextrecord': 32}
{'matches': 44, 'returned': 10, 'nextrecord': 43}
{'matches': 44, 'returned': 1, 'nextrecord': 0}
{'scheme': 'OPeNDAP:OPeNDAP', 'url': 'https://nbstds.met.no/thredds/dodsC/NBS/S1A/2025/03/16/IW/S1A_IW_GRDM_1SDV_20250316T062111_20250316T062138_058326_0735E5_95DE.nc'}
{'scheme': 'OGC:WMS', 'url': 'https://adc-wms.met.no/get_wms/7a91b2bf-e202-44c3-a89e-e7bdc79ec8e9/wms'}
{'scheme': 'WWW:DOWNLOAD-1.0-http--download', 'url': 'https://nbstds.met.no/thredds/fileServer/nbsArchive/S1A/2025/03/16/IW/S1A_IW_GRDM_1SDV_20250316T062111_20250316T062138_058326_0735E5_95DE.xml'}
{'scheme': 'OPeNDAP:OPeNDAP', 'url': 'https://nbstds.met.no/thredds/dodsC/NBS/S1A/2025/03/15/IW/S1A_IW_GRDM_1SDV_20250315T165625_20250315T165652_058318_073595_4DC0.nc'}
{'scheme': 'OGC:WMS', 'url': 'https://adc-wms.met.no/get_wms/1e294e5b-3d20-4948-b9d4-e678f615c8c8/wms'}
{'scheme